 # <center>**Modelo de recomendacion**

En este Jupyter se crea un modelo de ML de recomendacion de items por similitud del coseno es decir se ingresa un articulo y el modelo busca articulos similares y nos  recomienda segun su similitud, en este caso nos recomienda 5 juegos al ingresar el id de otro juego.

## Importaciones

In [43]:
import pandas as pd
import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Extraccion de la informacion

**Datos a utilizar**

In [80]:
Tabla_API= pd.read_parquet('Tabla_API.parquet')

In [81]:
Tabla_API

,item_id,item_name,playtime_forever,user_id,playtime_hours,recommend,sentiment_analysis,year_review,genres,developer,year_release
0,22200,Zeno Clash,271,76561197970982479,4.516667,True,1,2011,Action,ACE Team,2009
1,22200,Zeno Clash,271,76561197970982479,4.516667,True,1,2011,Indie,ACE Team,2009
2,22200,Zeno Clash,240,seantheextraprawnsheepguy,4.000000,True,2,2011,Action,ACE Team,2009
3,22200,Zeno Clash,240,seantheextraprawnsheepguy,4.000000,True,2,2011,Indie,ACE Team,2009
4,22200,Zeno Clash,690,pipekissXD,11.500000,True,2,2013,Action,ACE Team,2009
...,...,...,...,...,...,...,...,...,...,...,...
97519,220090,The Journey Down: Chapter One,403,MeloncraftLP,6.716667,True,1,2014,Indie,SkyGoblin,2013
97520,431510,Mystic Destinies: Serendipity of Aeons,499,vinquility,8.316667,True,2,2014,Adventure,Aeon Dream Studios,2016
97521,431510,Mystic Destinies: Serendipity of Aeons,499,vinquility,8.316667,True,2,2014,Casual,Aeon Dream Studios,2016
97522,431510,Mystic Destinies: Serendipity of Aeons,499,vinquility,8.316667,True,2,2014,Indie,Aeon Dream Studios,2016


Para entrenar el modelo solamente necesitamos 'item_id', 'item_name' asi que creamos una tabla con estas dos columnas

In [82]:
modelo= Tabla_API[['item_id', 'item_name', 'genres']]

In [83]:
#Visualizamos
modelo

,item_id,item_name,genres
0,22200,Zeno Clash,Action
1,22200,Zeno Clash,Indie
2,22200,Zeno Clash,Action
3,22200,Zeno Clash,Indie
4,22200,Zeno Clash,Action
...,...,...,...
97519,220090,The Journey Down: Chapter One,Indie
97520,431510,Mystic Destinies: Serendipity of Aeons,Adventure
97521,431510,Mystic Destinies: Serendipity of Aeons,Casual
97522,431510,Mystic Destinies: Serendipity of Aeons,Indie


In [84]:
# Se agrupar por item_name y concatena las entradas de la columna 'genres'
modelo = modelo.groupby('item_name').agg({'item_id': 'first', 'genres': lambda x: ', '.join(x)}).reset_index()


# Se visualiza la tabla
modelo

,item_name,item_id,genres
0,! That Bastard Is Trying To Steal Our Gold !,449940,"Action, Adventure, Casual, Indie"
1,"10,000,000",227580,"Action, Casual, Indie, RPG, Action, Casual, In..."
2,100% Orange Juice,282800,"Indie, Strategy, Indie, Strategy, Indie, Strat..."
3,12 Labours of Hercules,342580,"Casual, Strategy, Casual, Strategy"
4,12 Labours of Hercules II: The Cretan Bull,360640,"Casual, Strategy, Casual, Strategy, Casual, St..."
...,...,...,...
2647,resident evil 4 / biohazard 4,254700,"Action, Adventure, Action, Adventure, Action, ..."
2648,sZone-Online,316390,"Action, Free to Play, Massively Multiplayer, R..."
2649,the static speaks my name,387860,"Adventure, Free to Play, Indie, RPG, Adventure..."
2650,theHunter,253710,"Action, Adventure, Free to Play, Simulation, S..."


Verificamos los duplicados

In [86]:
duplicados = modelo.duplicated().any()

# Imprimir resultado
if duplicados:
    print("Hay filas duplicadas")
else:
    print("No hay filas duplicadas")

No hay filas duplicadas


Guardamos la informacion en un parquet

In [87]:
modelo.to_parquet('modelo.parquet')

Creamos un objeto CountVectorizar

In [88]:
counvec = CountVectorizer(max_features = 5000, stop_words="english")

Aplicamos el vectorizador a los datos y transformamos los datos en una matriz de recuento de terminos y se convierte en una matriz

In [89]:
counvec.fit_transform(modelo["genres"]).toarray().shape

(2652, 29)

Generamos los vectores a comparar 

In [90]:
vectores = counvec.fit_transform(modelo["genres"]).toarray()

In [91]:
# Se observan la forma de los vectores
vectores

array([[ 0,  1,  1, ...,  0,  0,  0],
       [ 0,  2,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0, 15, ...,  0,  0,  0],
       [ 0, 53, 53, ...,  0,  0,  0],
       [ 0, 15, 15, ...,  0,  0,  0]], dtype=int64)

Se calcula la similitud del coseno entre los vectores

In [92]:
cosine_sim = cosine_similarity(vectores)

In [93]:
cosine_sim[0]

array([1.        , 0.75      , 0.35355339, ..., 0.4472136 , 0.40824829,
       0.57735027])

Se ordenan la lista de tuplas basandose en la similitus coseno, pasando de la más similar a menos similar tomando 5 valores

In [94]:
sorted(list(enumerate(cosine_sim[0])), reverse=True, key=lambda x:x[1])[1:6]

[(133, 1.0), (217, 1.0), (230, 1.0), (277, 1.0), (303, 1.0)]

## Función de recomendación según un juego

Ahora, conociendo la relación entre los distintos juegos, se puede proponer una función que realice una recomendación de 5 juegos en función de un juego dado, teniendo en cuenta los valores mas altos de similitud del coseno.

In [96]:
def recomendacion(game):
    indice_juegos = modelo[modelo["item_name"]==game].index[0]
    distances = cosine_sim[indice_juegos]
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    recommended_titles = [modelo.iloc[i[0]]['item_name'] for i in lista_juegos]
    return recommended_titles

In [97]:
recomendacion('theHunter')

['Endless Sky',
 'S.K.I.L.L. - Special Force 2',
 'Back to Dinosaur Island ',
 'Half-Life 2: Update',
 'Portal Stories: Mel']

**Generación de una tabla de resultados**

Se aplica la función al dataframe para obtener una nueva columna con las recomendaciones ya que es más facil de cargar

In [98]:
modelo['recomendaciones'] = modelo['item_name'].apply(recomendacion)

In [99]:
modelo.head()

,item_name,item_id,genres,recomendaciones
0,! That Bastard Is Trying To Steal Our Gold !,449940,"Action, Adventure, Casual, Indie",[Aperture Tag: The Paint Gun Testing Initiativ...
1,"10,000,000",227580,"Action, Casual, Indie, RPG, Action, Casual, In...","[Castle Crashers, Forward to the Sky, Starboun..."
2,100% Orange Juice,282800,"Indie, Strategy, Indie, Strategy, Indie, Strat...","[AI War: Fleet Command, Castle Story, DEFCON, ..."
3,12 Labours of Hercules,342580,"Casual, Strategy, Casual, Strategy","[12 Labours of Hercules, Battle Chess: Game of..."
4,12 Labours of Hercules II: The Cretan Bull,360640,"Casual, Strategy, Casual, Strategy, Casual, St...","[12 Labours of Hercules, Battle Chess: Game of..."


Creamos un parquet para deployar en render la funcion de recomendacion, debido a la capacidad del mismo

In [100]:
modelo.to_parquet('recomendaciones.parquet')

Traemos el dataframe que usaremos ahora

In [101]:
recomendaciones= pd.read_parquet('recomendaciones.parquet')

Esta función sirve en el notebook pero no en la API por lo que se tuvo que cambiar y hacer otra que deployara en Render, sin ambargo realiza la misma funcion

In [102]:
def recomendacion_juego(id_producto):
    recomendacion = recomendaciones[recomendaciones['item_id'] == id_producto]['recomendaciones'].iloc[0]
    
    return {
        'Te recomendamos algunos juegos similares al ingresado': recomendacion
    }

In [103]:
recomendacion_juego(1250)

{'Te recomendamos algunos juegos similares al ingresado': array(['Alien Breed 2: Assault', 'Alien Rage - Unlimited',
        'Alien: Isolation', 'Aliens versus Predator Classic 2000',
        'Aliens vs. Predator'], dtype=object)}